# Notebook to iterate on Bokeh Plots

**Website for Bokeh Plot:** https://camilofosco.com/brain2video_viz_website


In [ ]:
import sys
import os
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append('../')
from utils import compute_tsne_embeddings, load_all_fmri_for_subject
import json
import base64

%load_ext autoreload
%autoreload 2


In [ ]:
# Load pkl file with moments data

pkl_name = "EBA_TRavg-56789_testing.pkl"
sub = "sub01"
with open(f"./data/{sub}/{pkl_name}", "rb") as f:
    data = pkl.load(f)

# Print information about loaded data
print(f"{pkl_name} loaded. Data is dict with keys: {data.keys()}")
print(f"Each key has shape: \ntrain_data: {data['train_data'].shape}, \ntest_data: {data['test_data'].shape}, \nnoise_ceiling: {data['noise_ceiling'].shape}, \np_values: {data['p_values'].shape}")
print(f"Dimensions correspond to num_videos, num_repetitions, num_voxels")

In [ ]:
# Load labels that we want to showcase for videos
labels = pd.read_csv('../data/labels_4_classes.csv')

#replace nans with zeros
labels = labels.fillna(0)

# define column names
col_names = ['video_id',
            'human_face_close_up', 
            'outdoor_natural_scene_no_humans', 
            'presence_of_humans', 
            'hands_close_up'
            ]

# Overwrite col names 
labels.columns = col_names


# set columns to integers
labels[col_names] = labels[col_names].astype(int)

# Ifall columsn are zero, put a one in new column "other"
labels['other'] = np.where((labels[col_names[1:]] == 0).all(axis=1), 1, 0)
labels


In [ ]:
# Encode first frames of each video in base64

import base64
from io import BytesIO
from PIL import Image

video_frames_folder = '../data/stimuli/frames'

image_bytes = []
# for f in sorted(os.listdir(video_frames_folder)):
#     first_frame = Image.open(os.path.join(video_frames_folder, f, "001.png")).convert('RGB')
#     buffer = BytesIO()
#     first_frame.save(buffer, format='JPEG')
#     image_byte = base64.b64encode(buffer.getvalue()).decode('utf-8')
#     image_bytes.append(image_byte)

In [ ]:
# Transform videos to gifs
%load_ext autoreload
%autoreload 2

from utils import transform_vids_to_gifs

# transform_vids_to_gifs(path_to_vids='../data/stimuli/mp4', 
#                        path_to_gifs='../data/stimuli/gif', 
#                        size=128,
#                        start_from=358)


In [ ]:
# Encode gifs in base64

gif_folder = './data/stimuli/gif'

gif_base64 = []
for f in sorted(os.listdir(gif_folder)):
    if f.endswith('.gif'):
        with open(os.path.join(gif_folder, f), 'rb') as f:
            gif_base64.append(base64.b64encode(f.read()).decode('ascii'))




## Compute Embeddings

In [ ]:
# Compute t-sne for all brain areas and save data in json format

sub = 'sub01'
fmri_data = load_all_fmri_for_subject("./data/"+sub)

os.makedirs(f'./data/tsne_results/{sub}', exist_ok=True)
# for brain_area, data in fmri_data.items():
#     tsne_results = compute_tsne_embeddings(data, perplexity=95, n_iter=1000)
#     tsne_results = tsne_results.tolist()
#     with open(f'./data/tsne_results/{sub}/{brain_area}.json', 'w') as f:
#         json.dump(tsne_results, f)
    


In [ ]:
# Transfrom ben's new tsne into a json containing a simple array of 2d points

def tsne_reformat_dict_to_array(dict_tsne_path, arr_tsne_path):

    for j in os.listdir(dict_tsne_path):

        subject = j.split("_")[0]
        brain_area_name = j.split("_")[1].split("ROI-")[-1]
        with open(os.path.join(dict_tsne_path, j)) as f:
            data = json.load(f)

        arr = [[d["tsne1"], d["tsne2"]] for d in data.values()]

        # Save as json with format subject/brain_area_name.json
        save_path = os.path.join(arr_tsne_path, subject)
        if not os.path.exists(save_path): os.makedirs(save_path)
        with open(os.path.join(save_path, f"{brain_area_name}.json"), "w") as f:
            json.dump(arr, f)

    
# tsne_reformat_dict_to_array('./data/new_tsnes_from_ben_as_dict/', './data/new_tsnes_from_ben_as_arr')



### Build Bokeh plot to visualize fMRI responses in 2D
Use T-SNE, PCA and UMap

In [ ]:
# Generate bokeh viz

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Select, CustomJS, WheelZoomTool
from bokeh.io import output_notebook
from bokeh.layouts import row, column


concepts = list(labels.columns[1:])
def get_colors(concepts):
    colors = []
    for i in range(len(concepts)):
        if concepts[i] == 'human_face_close_up':
            colors.append([255,0,0,1])
        elif concepts[i] == 'outdoor_natural_scene_no_humans':
            colors.append([0,255,0,1])
        elif concepts[i] == 'presence_of_humans':
            colors.append([139,69,19,1])
        elif concepts[i] == 'hands_close_up':
            colors.append([0,0,255,1])
        elif concepts[i] == 'other':
            colors.append([0,0,0,0.1])
        else:
            # append random color
            colors.append([np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255), 1])
    return colors

colors_rgba = get_colors(concepts)

# Assign a color to each datapoint in the dataframe
color_lst = []
for i in range(len(labels)):
    for j in range(len(concepts)):
        if labels[concepts[j]][i] == 1:
            color_lst.append(colors_rgba[j])
            break


# Assign a class name to each datapoint in the dataframe
class_names = []
for i in range(len(labels)):
    for j in range(len(concepts)):
        if labels[concepts[j]][i] == 1:
            class_names.append(concepts[j])
            break

# Load tsne results from json as numpy array

def load_tsne_results(path):
    
    with open(path, 'r') as f:
        tsne_results = np.array(json.load(f))

    return tsne_results


data_dir = './data/new_tsnes_from_ben_as_arr/'

starting_area = 'EBA'
starting_subject = 'allSubjects'
starting_tsne = load_tsne_results(os.path.join(data_dir, starting_subject, f'{starting_area}.json'))

source = ColumnDataSource(
            data={'x': starting_tsne[:, 0], 
                 'y': starting_tsne[:, 1], 
                 'color': color_lst, 
                 'class': class_names, 
                 # 'image': image_bytes[:1000], 
                 'image': gif_base64, 
                 'video_number': np.arange(1,len(starting_tsne)+1),
                 # 'video': video_base64[:1000],
                 })
fig = figure(tools='wheel_zoom, pan, box_zoom, reset')
# activate the wheel zoom tool
fig.toolbar.active_scroll = fig.select_one(WheelZoomTool)
fig.scatter(x='x', y='y', size=12, fill_color='color', line_color=None,
            source=source, legend_field='class')

# Add dropdown to change brain areas
# BRAIN_AREAS = sorted([b.split('.')[0] for b in os.listdir(os.path.join(data_dir, starting_subject)) if 'json' in b])
BRAIN_AREAS = "V1v, V1d, V2v, V2d, V3v, V3d, hV4, LOC, EBA, OFA, FFA, STS, PPA, RSC, TOS, V3ab, IPS0, IPS1-2-3, 7AL, BA2, PFt, PFop".split(', ')
brain_area_dropdown = Select(title="Brain area", value = starting_area, options=BRAIN_AREAS)


# Add dropdown to change subject
SUBJECT_OPTIONS = sorted([s for s in os.listdir(data_dir)])
subject_dropdown = Select(title="Subject", value = starting_subject, options = SUBJECT_OPTIONS)


brain_area_dropdown.js_on_change('value',
    CustomJS(args=dict(source=source, subject_dropdown=subject_dropdown), code="""
        var data = source.data;
        var new_area = cb_obj.value;
        var current_subject = subject_dropdown.value;
        var xhr = new XMLHttpRequest();
        xhr.open('GET', `""" 
        + data_dir + 
        """${current_subject}/${new_area}.json`);
        xhr.onload = function() {
            var tsne = JSON.parse(xhr.responseText);
            data['x'] = tsne.map(function(x) { return x[0]; });
            data['y'] = tsne.map(function(x) { return x[1]; });
            source.change.emit();
        }
        xhr.send();
        """)
)

subject_dropdown.js_on_change('value',
    CustomJS(args=dict(source=source, area_dropdown=brain_area_dropdown), code="""
    var data = source.data;
    var new_subject = cb_obj.value;
    var current_area = area_dropdown.value;
    var xhr = new XMLHttpRequest();
    xhr.open('GET', `""" 
    + data_dir + 
    """${new_subject}/${current_area}.json`);
    xhr.onload = function() {
        var tsne = JSON.parse(xhr.responseText);
        data['x'] = tsne.map(function(x) { return x[0]; });
        data['y'] = tsne.map(function(x) { return x[1]; });
        source.change.emit();
    }
    xhr.send();
    """)
)

hover = HoverTool(
    tooltips=
            '<p>video ID: @video_number</p>' +
            '<img src="data:image/gif;base64,@image" width="128" height="128">'
)   
fig.add_tools(hover)

widgets = row(brain_area_dropdown, subject_dropdown)
main_layout = column(widgets, fig)


output_notebook()
show(main_layout) 

## Insert bokeh viz into website 

In [ ]:
from bokeh.resources import CDN
from bokeh.embed import file_html

# Generate html from the figure, then insert in brain2video_viz_website/index.html, 
# replacing the current scripts


html = file_html(main_layout, CDN, "Embeddings visualization of fMRI data")
with open('bokeh_plot.html', 'w') as f:
    f.write(html)

    
